# LSTM Baseline Model Training

Este notebook treina modelos LSTM baseline (sem técnicas de privacidade) para ambos os datasets.

## O que este notebook faz:
1. Carrega dados processados (Sleep-EDF e WESAD)
2. Treina modelo LSTM baseline para cada dataset
3. Avalia performance em validação e teste
4. Salva modelos e resultados
5. Cria visualizações do treino

**Pré-requisitos**: Execute os notebooks 00, 01 e 02 primeiro.


In [ ]:
# ============================================================================
# STEP 1: Setup and Load Data
# ============================================================================

print("="*70)
print("LSTM BASELINE TRAINING")
print("="*70)

# Import modules
from src.models.lstm_baseline import train_baseline, evaluate_model, save_model, get_default_config
from src.preprocessing.sleep_edf import load_processed_sleep_edf
from src.preprocessing.wesad import load_processed_wesad
from src.evaluation.visualization import plot_training_history
import numpy as np
import os

# Define paths
SLEEP_DATA_PATH = '/content/drive/MyDrive/mhealth-data/processed/sleep-edf'
WESAD_DATA_PATH = '/content/drive/MyDrive/mhealth-data/processed/wesad'
MODELS_PATH = '/content/drive/MyDrive/mhealth-data/models'
RESULTS_PATH = '/content/drive/MyDrive/mhealth-data/results'

print("Loading processed data...")

# Load Sleep-EDF data
if os.path.exists(SLEEP_DATA_PATH):
    X_train_sleep, X_val_sleep, X_test_sleep, y_train_sleep, y_val_sleep, y_test_sleep, _, _, sleep_info = load_processed_sleep_edf(SLEEP_DATA_PATH)
    print(f"✅ Sleep-EDF loaded: {X_train_sleep.shape}")
else:
    print("❌ Sleep-EDF data not found. Run notebook 01 first.")
    X_train_sleep = None

# Load WESAD data
if os.path.exists(WESAD_DATA_PATH):
    X_train_wesad, X_val_wesad, X_test_wesad, y_train_wesad, y_val_wesad, y_test_wesad, _, _, wesad_info = load_processed_wesad(WESAD_DATA_PATH)
    print(f"✅ WESAD loaded: {X_train_wesad.shape}")
else:
    print("❌ WESAD data not found. Run notebook 02 first.")
    X_train_wesad = None


In [ ]:
# ============================================================================
# STEP 2: Train Sleep-EDF Baseline Model
# ============================================================================

if X_train_sleep is not None:
    print("\n" + "="*70)
    print("TRAINING SLEEP-EDF BASELINE MODEL")
    print("="*70)
    
    # Get default configuration
    config = get_default_config()
    config.update({
        'dataset': 'sleep_edf',
        'model_type': 'baseline',
        'privacy_technique': 'None'
    })
    
    print(f"Configuration: {config}")
    
    # Train model
    model_sleep, history_sleep = train_baseline(
        X_train_sleep, y_train_sleep,
        X_val_sleep, y_val_sleep,
        config
    )
    
    # Evaluate model
    results_sleep = evaluate_model(model_sleep, X_test_sleep, y_test_sleep, config['window_size'])
    results_sleep.update({
        'dataset': 'sleep_edf',
        'model_type': 'baseline',
        'privacy_technique': 'None',
        'privacy_parameter': 'N/A'
    })
    
    # Save model and results
    save_model(
        model_sleep, history_sleep, results_sleep,
        MODELS_PATH, 'sleep_edf_baseline'
    )
    
    # Save training history plot
    plot_training_history(
        history_sleep.history,
        save_path=f'{RESULTS_PATH}/training_history_sleep_edf_baseline.png',
        title='Sleep-EDF Baseline Training History'
    )
    
    print(f"\n✅ Sleep-EDF baseline model trained successfully!")
    print(f"Test Accuracy: {results_sleep['accuracy']:.4f}")
    print(f"Test F1-Score: {results_sleep['f1_score']:.4f}")
    
else:
    print("❌ Skipping Sleep-EDF training - data not available")


In [ ]:
# ============================================================================
# STEP 3: Train WESAD Baseline Model
# ============================================================================

if X_train_wesad is not None:
    print("\n" + "="*70)
    print("TRAINING WESAD BASELINE MODEL")
    print("="*70)
    
    # Get default configuration
    config = get_default_config()
    config.update({
        'dataset': 'wesad',
        'model_type': 'baseline',
        'privacy_technique': 'None'
    })
    
    print(f"Configuration: {config}")
    
    # Train model
    model_wesad, history_wesad = train_baseline(
        X_train_wesad, y_train_wesad,
        X_val_wesad, y_val_wesad,
        config
    )
    
    # Evaluate model
    results_wesad = evaluate_model(model_wesad, X_test_wesad, y_test_wesad, config['window_size'])
    results_wesad.update({
        'dataset': 'wesad',
        'model_type': 'baseline',
        'privacy_technique': 'None',
        'privacy_parameter': 'N/A'
    })
    
    # Save model and results
    save_model(
        model_wesad, history_wesad, results_wesad,
        MODELS_PATH, 'wesad_baseline'
    )
    
    # Save training history plot
    plot_training_history(
        history_wesad.history,
        save_path=f'{RESULTS_PATH}/training_history_wesad_baseline.png',
        title='WESAD Baseline Training History'
    )
    
    print(f"\n✅ WESAD baseline model trained successfully!")
    print(f"Test Accuracy: {results_wesad['accuracy']:.4f}")
    print(f"Test F1-Score: {results_wesad['f1_score']:.4f}")
    
else:
    print("❌ Skipping WESAD training - data not available")


In [ ]:
# ============================================================================
# STEP 4: Summary and Next Steps
# ============================================================================

print("\n" + "="*70)
print("BASELINE TRAINING COMPLETE!")
print("="*70)

# Collect all results
all_results = {}

if X_train_sleep is not None:
    all_results['Sleep-EDF Baseline'] = results_sleep
    print(f"Sleep-EDF Baseline - Accuracy: {results_sleep['accuracy']:.4f}, F1: {results_sleep['f1_score']:.4f}")

if X_train_wesad is not None:
    all_results['WESAD Baseline'] = results_wesad
    print(f"WESAD Baseline - Accuracy: {results_wesad['accuracy']:.4f}, F1: {results_wesad['f1_score']:.4f}")

# Save combined results
if all_results:
    from src.evaluation.metrics import save_evaluation_results
    save_evaluation_results(all_results, RESULTS_PATH, 'baseline_results.json')

print(f"\nModels saved to: {MODELS_PATH}")
print(f"Results saved to: {RESULTS_PATH}")

print("\nNext steps:")
print("1. Run notebook 04_train_dp.ipynb for Differential Privacy")
print("2. Run notebook 05_train_fl.ipynb for Federated Learning")
print("3. Run notebook 06_analysis.ipynb for final analysis")

print("\n✅ Baseline models are ready for comparison with privacy-preserving techniques!")
